In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import function
import pandasql
import sqlite3
from IPython.display import clear_output



In [2]:
df = pd.read_excel('VisitorEntrancesV3.xlsx')

In [3]:
df.head()

,Date,Site_Name,Israelis_Count,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,is_camping,special_activity,...,Season_summer,Season_winter,last_year_visitors,Exceeded_pm10_in_Israel,Site_Exceeded,IsHoliday,IsVacation,IsHeritage,Temperature,isHeatwave
0,2016-03-02,Akhziv,211,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,30.1,0
1,2016-03-17,Akhziv,186,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,19.2,0
2,2016-03-21,Akhziv,59,0,0,1,0,0,1,0,...,0,0,NaN,0,0,False,False,False,20.1,0
3,2016-03-25,Akhziv,6,0,0,1,0,1,1,0,...,0,0,NaN,1,0,True,True,False,26.5,0
4,2016-04-01,Akhziv,9,0,0,1,0,1,1,0,...,0,0,NaN,0,0,False,True,False,27.7,0


In [ ]:
tmp_df = df[['Date','Site_Name','Israelis_Count','Site_Exceeded']]
tmp_df

In [21]:
totalDays = len(tmp_df.pivot_table(index='Date'))
totalDaysExceeded = tmp_df.pivot_table(index=['Date','Site_Name']).Site_Exceeded.sum()

tmp_df.pivot_table(index=['Site_Name','Date']).Site_Exceeded

Site_Name  Date      
Akhziv     2016-03-02    0
           2016-03-17    0
           2016-03-21    0
           2016-03-25    0
           2016-04-01    0
                        ..
Yehudiya   2019-12-23    1
           2019-12-24    0
           2019-12-27    0
           2019-12-28    0
           2019-12-30    0
Name: Site_Exceeded, Length: 75165, dtype: int64

זיהום בישראל בערים יום לפני לעומת יום אחרי

In [22]:
def testPmSite(df,site):
    dataset = df
    dataset = df.loc[(df.Site_Name==site)&(df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(site)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==0 and now== 1 :
            l_EX_NOT.append(tmp['Israelis_Count'][i-1])
            L_EX.append(tmp['Israelis_Count'][i])




    print('length of paired data\n',len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean\n',np.mean(L_EX))
    print('EX median\n',np.median(L_EX))
    print('EX std\n',np.std(L_EX))
    print('EX_Not mean\n',np.mean(l_EX_NOT))
    print('EX_Not median\n',np.median(l_EX_NOT))
    print('EX_Not std\n',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}


pm10SiteTest = pd.DataFrame()
for site in tmp_df.Site_Name.unique():
    pm10SiteTest = pm10SiteTest.append(testPmSite(df,site),ignore_index=True)
clear_output(wait=True)
pm10SiteTest

,Site,Data Size,P_Value,Significant
0,Akhziv,39,0.61,0
1,Amud Stream,94,0.40,0
2,Apollonia,70,0.30,0
3,Arbel,101,0.09,0
4,Ashkelon National Park,54,0.89,0
5,Avdat,62,0.08,0
6,Ayun Stream,60,0.11,0
7,Baram,104,0.02,1
8,Bet Alpha,57,0.71,0
9,Bet Guvrin,57,0.12,0


In [23]:
def testPmAllSite(df):
    dataset = df
    site = 'All Site'
    dataset = df.loc[(df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==0 and now== 1 :
            l_EX_NOT.append(tmp['Israelis_Count'][i-1])
            L_EX.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSite(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(46566, 32)
length of paired data 3972 3972
EX mean 271.49144008056396
EX median 100.0
EX std 462.75908197777284
EX_Not mean 290.96852970795567
EX_Not median 110.0
EX_Not std 478.54073958509736
pval -  0.00534062 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_14416\4183378276.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSite(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,All Site,3972,0.01,1


In [24]:
def testPmAllSiteNorth(df):
    dataset = df
    site = 'region_North'
    dataset = df.loc[(df.region_North==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==0 and now== 1 :
            l_EX_NOT.append(tmp['Israelis_Count'][i-1])
            L_EX.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSiteNorth(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(26998, 32)
length of paired data 2178 2178
EX mean 222.6795224977043
EX median 84.0
EX std 363.5810767988344
EX_Not mean 250.06611570247935
EX_Not median 95.0
EX_Not std 414.9252969591387
pval -  0.00996401 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_14416\4120574490.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSiteNorth(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_North,2178,0.01,1


In [25]:
def testPmAllSiteSouth(df):
    dataset = df
    site = 'region_South Site'
    dataset = df.loc[(df.region_South==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==0 and now== 1 :
            l_EX_NOT.append(tmp['Israelis_Count'][i-1])
            L_EX.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSiteSouth(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(9438, 32)
length of paired data 847 847
EX mean 453.2833530106257
EX median 146.0
EX std 705.9059340420605
EX_Not mean 452.38724911452186
EX_Not median 134.0
EX_Not std 692.3712714627738
pval -  0.67609499 - no differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_14416\608390346.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSiteSouth(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_South Site,847,0.68,0


In [49]:
def testPmAllSiteCenter(df):
    dataset = df
    site = 'region_Central Site'
    dataset = df.loc[(df.region_Central==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==0 and now== 1 :
            l_EX_NOT.append(tmp['Israelis_Count'][i-1])
            L_EX.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSiteCenter(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(7016, 43)
length of paired data 209 209
EX mean 285.8086124401914
EX median 143.0
EX std 431.05880738968574
EX_Not mean 392.6842105263158
EX_Not median 161.0
EX_Not std 573.6439435409554
pval -  0.01084755 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_6624\614104063.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSiteCenter(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_Central Site,209,0.01,1


In [42]:
def testPmAllSite_Judea_Samaria(df):
    dataset = df
    site = 'region_Judea_Samaria Site'
    dataset = df.loc[(df.region_Judea_Samaria==1)& (df.IsHoliday==0)&(df.IsVacation==0)]
    tmp = dataset.sort_values(['Site_Name','Date']).reset_index()
    print(tmp.shape)
    L_EX = []
    l_EX_NOT = []
    for i in range(1,len(tmp)):    
        prev = tmp['Date'][i-1]
        now = tmp['Date'][i]
        if not function.date_diff(prev,now,1):
            continue    
        prev = tmp['Site_Name'][i-1]
        now = tmp['Site_Name'][i]
        if now!=prev:
            continue
        prev = tmp['Site_Exceeded'][i-1]
        now = tmp['Site_Exceeded'][i]
        if prev==0 and now== 1 :
            l_EX_NOT.append(tmp['Israelis_Count'][i-1])
            L_EX.append(tmp['Israelis_Count'][i])




    print('length of paired data',len(L_EX),len(l_EX_NOT))

    data_size = len(L_EX)
    if data_size<1 :
        return {"Site":site,"Data Size":"No Data","P_Value":"No Data","Significant":'No Data'}
    print('EX mean',np.mean(L_EX))
    print('EX median',np.median(L_EX))
    print('EX std',np.std(L_EX))
    print('EX_Not mean',np.mean(l_EX_NOT))
    print('EX_Not median',np.median(l_EX_NOT))
    print('EX_Not std',np.std(l_EX_NOT))
    stat, p = stats.wilcoxon(l_EX_NOT, L_EX,alternative='greater')
    Significant=0
    if p > 0.05:
        print('pval - ',"%.8f" %p, '- no differnce')
    else:
        Significant=1
        print('pval - ',"%.8f" %p, '- differnce')
    
    return  {"Site":site,"Data Size":data_size,"P_Value":round(p,2),"Significant":Significant}

    # pd.DataFrame(data={
    #     'Day_before_exceeded':L_EX,
    #     'Day_after_not_exceeded':l_EX_NOT,

    # }).to_csv('exceeded_Pm10_In_Site.csv')

pm10SiteTest = pd.DataFrame()
pm10SiteTest = pm10SiteTest.append(testPmAllSite_Judea_Samaria(df),ignore_index=True)
# clear_output(wait=True)
pm10SiteTest

(5860, 43)
length of paired data 183 183
EX mean 113.56284153005464
EX median 65.0
EX std 158.87867019613373
EX_Not mean 147.0655737704918
EX_Not median 78.0
EX_Not std 203.5472745281552
pval -  0.00020588 - differnce


C:\Users\Avivsa\AppData\Local\Temp\ipykernel_6624\1122883092.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pm10SiteTest = pm10SiteTest.append(testPmAllSite_Judea_Samaria(df),ignore_index=True)


,Site,Data Size,P_Value,Significant
0,region_Judea_Samaria Site,183,0.0,1
